In [2]:
# Draw the current state of the game
def drawboard(board):
    print("Current state: \n\n");
    for i in range (0,9):
        if board[i] == 0:           # If value is 0, it's an empty cell
            print("- ",end = " ");
        if board[i] == 1:           # If value is 1, it's a mark of AI
            print("X ",end = " ");
        if board[i] == -1:          # If value is -1, it's a mark of the human
            print("O ",end = " ");
        if (i+1) % 3 == 0:          # Move to the next line after printing 3 elements
            print("\n");            
    print("\n");  # Move the next line after printing the game board

# Check if there is a player wins.
# If human player wins, return -1. 
# If AI player wins, return 1. 
# If draw, return 0.
# If the game is not finished, return 2.
def check_game(board):
    # There are 8 possible winning patterns.
    # 0, 1, 2 (1st row), 3, 4, 5 (2nd row), 6, 7, 8 (3rd row)
    # 0, 3, 6 (1st col), 1, 4, 7 (2nd col), 2, 5, 8 (3rd col)
    # 0, 4, 8 (Diagonal - top-left to bottom-right)
    # 2, 4, 6 (Diagonal - top-right to bottom-left)
    winning_pos = [ [0, 1, 2], [3, 4, 5], [6, 7, 8],
                    [0, 3, 6], [1, 4, 7], [2, 5, 8],
                    [0, 4, 8], [2, 4, 6] ]

    # Check all possible winning patterns for each player.
    for i in range(0,8):
        # Check whether the cell is non-empty AND the winning positions with the same mark
        if board[winning_pos[i][0]] != 0 and\
           board[winning_pos[i][0]] == board[winning_pos[i][1]] and\
           board[winning_pos[i][0]] == board[winning_pos[i][2]]:
           # Fulfiled the winning condition, return the mark of player
           return board[winning_pos[i][2]];

    # When reaching here, it means no player wins yet
    # Check whether there is an empty cell (i.e. with 0).
    # If so, the game is not finished. Return 2
    if any(element == 0 for element in board):
       return 2
    
    return 0  # Reach here, it means the game draws

# Obtain the user's input and update the board
def human_turn(board):
    pos = input("Enter O's position [1 to 9]: ");
    pos = int(pos); # Obtain user input, i.e., where to put the mark

    # If the cell picked by the human player is non-empty (i.e., non-zero), illegal
    if board[pos-1] != 0:
       print("Illegal Move!")
       exit(1)      # Terminate the program

    # When reaching here, the move is legal and we put -1 (put the mark of human player)
    board[pos-1] = -1

# Perform minimax search with alpha-beta pruning
def minimax_abp(board, player, alpha = -float('inf'), beta = float('inf')):
    global count                # Refer to the global variable count in main
    count += 1                  # Make 1 call of minimax_abp already
    result = check_game(board)  # Check if any player wins
    if result != 2:             # If the result is not 2, the game is finished,
       return result            # return 1 if the winner is AI,
                                # otherwise return -1 (i.e., human is the winner)
    if player == 1:
       value = -float('inf')
       for i in range(0,9):              # Check all board locations           
           if board[i] == 0:             # If the cell is empty
              board[i] = player          # Try to put the player's mark at cell i+1
              # Perform minimax with alpha-beta pruning for the next player 
              # and update current value if needed
              value = max(value, minimax_abp(board, player*-1, alpha, beta))
              alpha = max(alpha, value)  # Update alpha with max of current alpha and value
              board[i] = 0               # Undo the trial
              if alpha >= beta: break
    else:
       value = float('inf')
       for i in range(0,9):              # Check all board locations
           if board[i] == 0:             # If the cell is empty
              board[i] = player          # Try to put the player's mark at cell i+1
              # Perform minimax with alpha-beta pruning for the next player
              # and update current value if needed
              value = min(value, minimax_abp(board, player*-1, alpha, beta))
              beta = min(beta, value)    # Update beta with min of current beta and value
              board[i] = 0               # Undo the trial
              if alpha >= beta: break
              
    # Return the max score if the player is AI or
    # return the min score if the player is human
    return value

# This function makes computer's move based on minimax.
def ai_turn(board):
    pos = -1;        # Initialize pos to illegal value, -1, here
    max_value = -2;  # Initialize max value so far to -2, 
                     # which is a value smaller than the min value possible

    for i in range(0,9):
        if board[i] == 0:                  # If the cell is empty
           board[i] = 1                    # Try to put X at cell i+1
           score = minimax_abp(board, -1); # Calculate minimax score for the human player
           board[i] = 0                    # Undo the trial
           if score > max_value:           # If we can a better score in the next level,
              max_value = score            # update the score and pos
              pos = i
 
    board[pos] = 1                         # Put X at pos, which is the best move


# Main function

# The broad is represented in a single dimensional list
# Initalize the board to all zeros (i.e., all empty cells)
board = [ 0, 0, 0, 0, 0, 0, 0, 0, 0 ]

print("Computer: X VS. You: O")
first = input("Play first (Y/N) :")
if first == 'Y' or first == 'y': # If first is 'Y'/'y', human wants to play first
   player = 1
else:
   player = 0

for i in range (0,9):          # Play the same. 9 turns in total
    if check_game(board) != 2: # If the value returned by check_win is not 2, the game is finished
       break
    if (i+player) % 2 == 0:    # Take turns to play the game
       count = 0               # Count the number of minimax_abp calls
       ai_turn(board)
       print("Count:", count)  # Print the count
    else:
       drawboard(board)
       human_turn(board)
      
result = check_game(board)     # Check the game again

if result==0:                  # If the result is 0, draw
   drawboard(board)
   print("Draw!!!")
if result==1:                  # If the result is 1, AI wins
   drawboard(board)
   print("AI(X) Wins! Human(O) Loses!")   
if result==-1:                 # If the result is -1, human wins
   drawboard(board)
   print("Human(O) Wins! AI(X) Loses!")

Computer: X VS. You: O
Play first (Y/N) :y
Current state: 


-  -  -  

-  -  -  

-  -  -  



Enter O's position [1 to 9]: 1
Count: 4089
Current state: 


O  -  -  

-  X  -  

-  -  -  



Enter O's position [1 to 9]: 2
Count: 220
Current state: 


O  O  X  

-  X  -  

-  -  -  



Enter O's position [1 to 9]: 4
Count: 24
Current state: 


O  O  X  

O  X  -  

X  -  -  



AI(X) Wins! Human(O) Loses!
